In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load dataset
dataset_path = "austinHousingData.csv"
df = pd.read_csv(dataset_path)

# Define target variable
target_column = "latestPrice"

# Drop unnecessary columns
drop_columns = ["zpid", "city", "streetAddress", "description", "homeImage", "latestPriceSource", 
                "latest_saledate", "latest_salemonth", "latest_saleyear"]
df.drop(columns=[col for col in drop_columns if col in df.columns], inplace=True)

# Handle outliers
df = df[df['latitude'] >= 30.12]
df = df[df['numOfBathrooms'] > 0]
df = df[df['numOfBedrooms'] > 0]
df = df[df[target_column] <= 3000000]
df = df[df[target_column] > 75000]
df = df[df['zipcode'] != 78734]

# Encode categorical variables
df = pd.get_dummies(df, drop_first=True)

# Split dataset into features and target variable
X = df.drop(columns=[target_column])
y = df[target_column]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train ensemble regression models
models = {
    "XGBoost": xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100, random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42)
}

# Train and evaluate models
results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    results[name] = {
        "MAE": mean_absolute_error(y_test, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        "R² Score": r2_score(y_test, y_pred)
    }

# Convert results to DataFrame
results_df = pd.DataFrame(results).T

# Display results
print(results_df)


                            MAE           RMSE  R² Score
XGBoost            76623.272302  140446.888901  0.809899
Random Forest      74812.087425  137261.729290  0.818423
Gradient Boosting  82949.456405  141084.960640  0.808167
